<a href="https://colab.research.google.com/github/SairajNeelam/CoronaVirus-Tweet-Sentiment-Analysis/blob/main/CoronaVirus_Tweet_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***Problem Statement***

> **We have to build a classification model to predict the sentiment of Covid-19 tweets**.





### **What is sentiment analysis?**
**Sentiment analysis, also known as opinion mining, or emotion AI, boils down to one thing:**

> **It’s the process of analyzing online pieces of writing to determine the emotional tone they carry, whether they’re positive, negative, or neutral.**

>**In simple words, sentiment analysis helps to find the author’s attitude towards a topic. Sentiment analysis tools categorize pieces of writing as positive, neutral, or negative**.

### **Why sentiment analysis is important?**

1. **Sentiment analysis is important because emotions and attitudes towards a topic can become actionable pieces of information useful in numerous areas of business and research.**
2. **Text analytics and opinion mining find numerous applications in e-commerce, marketing, advertising, politics, market research and any other research, which is very helpful for businesses.**
3. **It saves time and effort. Assessing the tone they carry manually is doable, but takes ages.**